In [1]:
import pandas as pd
import base64
from io import BytesIO
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load CLIP model and processor
device = "cuda"
torch_dtype = torch.float16
model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32', device_map = device, torch_dtype=torch_dtype)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# **`Define categories with prompts`**


# Load CLIP model and processor

In [ ]:
# Define categories with prompts
categories = {
    "safe for children": [
        "This image is suitable for children.",
        "This image contains friendly and educational content.",
        "This image is child-appropriate.",
        "This image contains no harmful or inappropriate content."
    ],
    "violence": [
        "This image depicts violence.",
        "This image contains weapons.",
        "This image depicts fights or bullying.",
        "This image contains blood or gore."
    ],
    "sexual content": [
        "This image contains nudity.",
        "This image is sexually explicit.",
        "This image is pornographic.",
        "This image contains inappropriate sexual content."
    ],
    "scary or shocking": [
        "This image is scary or disturbing.",
        "This image depicts horror or fear.",
        "This image is dark and creepy.",
        "This image is shocking or disgusting."
    ]
}

prompts = []
labels = []
for category, descriptions in categories.items():
    prompts.extend(descriptions)
    labels.extend([category] * len(descriptions))
print(prompts, labels)

['This image is suitable for children.', 'This image contains friendly and educational content.', 'This image is child-appropriate.', 'This image contains no harmful or inappropriate content.', 'This image depicts violence.', 'This image contains weapons.', 'This image depicts fights or bullying.', 'This image contains blood or gore.', 'This image contains nudity.', 'This image is sexually explicit.', 'This image is pornographic.', 'This image contains inappropriate sexual content.', 'This image is scary or disturbing.', 'This image depicts horror or fear.', 'This image is dark and creepy.', 'This image is shocking or disgusting.'] ['safe for children', 'safe for children', 'safe for children', 'safe for children', 'violence', 'violence', 'violence', 'violence', 'sexual content', 'sexual content', 'sexual content', 'sexual content', 'scary or shocking', 'scary or shocking', 'scary or shocking', 'scary or shocking']


# Function to classify url image

In [4]:
def classify_image(image_path):
    prompts = ['This image is suitable for children.', 'This image contains friendly and educational content.',
               'This image is child-appropriate.', 'This image contains no harmful or inappropriate content.',
               'This image depicts violence.', 'This image contains weapons.', 'This image depicts fights or bullying.',
               'This image contains blood or gore.', 'This image contains nudity.', 'This image is sexually explicit.',
               'This image is pornographic.', 'This image contains inappropriate sexual content.',
               'This image is scary or disturbing.', 'This image depicts horror or fear.', 'This image is dark and creepy.',
               'This image is shocking or disgusting.']
    labels = ['safe for children', 'safe for children', 'safe for children', 'safe for children',
              'violence', 'violence', 'violence', 'violence',
              'sexual content', 'sexual content', 'sexual content', 'sexual content',
              'scary or shocking', 'scary or shocking', 'scary or shocking', 'scary or shocking']


    url = image_path
    image = Image.open(requests.get(url, stream=True).raw)

    device = "cuda"

    inputs = processor(text=prompts, images=image, return_tensors="pt", padding=True)
    inputs.to(device)

    # Model Inference
    with torch.no_grad():
        with torch.autocast(device):
            outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

    category_probs_sum = {}
    for i, label in enumerate(labels):
        category_probs_sum[label] = category_probs_sum.get(label, 0) + probs[0][i].item()

    # Find the best category
    most_likely_label = max(category_probs_sum, key=category_probs_sum.get)
    confidence = category_probs_sum[most_likely_label]

    return most_likely_label, confidence, category_probs_sum

In [5]:
a,b,c = classify_image("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSBUjeNOx_hO6GewthgtiMgWwD21kt6M1rBgpC97KQaJJQFUfWG&s")
print(a,b,c)

sexual content 0.357696533203125 {'safe for children': 0.352691650390625, 'violence': 0.16217041015625, 'sexual content': 0.357696533203125, 'scary or shocking': 0.1274566650390625}


# Classify the images from Kiddle

In [6]:

image_csv = "/content/drive/MyDrive/Algorithm auditing/kiddle_rechecked_data_combined.csv"
data = pd.read_csv(image_csv)
labels = []
confidences = []
categories_probs = []

for i, row in data.iterrows():
    # handle NA values
    if i / 10 == 0:
        print(f"Processing row {i}")
    if pd.isna(row["Image URL"]):
        # all three new columns need to be handled!!!!!
        labels.append("NA")
        confidences.append("NA")
        categories_probs.append("NA")
    # if no NA, classify then
    else:
        try:
            image_URL = row["Image URL"]
            label, confidence, category_probs = classify_image(image_URL)

            labels.append(label)
            confidences.append(confidence)
            categories_probs.append(category_probs)
        except Exception as e:
            print(f"Error processing image at index {i}: {e}")
            labels.append("Error")
            confidences.append(0)
            categories_probs.append({})

print(f"Data looping finished")

# Add results to the DF
data["most_likely_label"] = labels
data["confidence"] = confidences
data["categories_probs"] = categories_probs

# save the updates Df to a new csv file
output_csv_path = "/content/drive/MyDrive/Algorithm auditing/kiddle_rechecked_data_combined_with_labels.csv"
data.to_csv(output_csv_path, index=False)


Processing row 0
Data looping finished


In [8]:
len(data), len(labels), len(confidences), len(categories_probs)

(3011, 3011, 3011, 3011)